# Web Scraping e Geração de PDF - Tutorial

Este notebook demonstra como realizar web scraping e gerar PDFs usando Python.

In [ ]:
# Importando as bibliotecas necessárias
from bs4 import BeautifulSoup
import requests
import json
import pyautogui
from datetime import datetime
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle

## 1. Web Scraping

Vamos criar funções para extrair dados de websites.

In [ ]:
def web_scraping(url):
    """Realiza web scraping em uma URL específica."""
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extrai dados
        data = {
            'title': soup.title.string if soup.title else '',
            'paragraphs': [p.text for p in soup.find_all('p')],
            'links': [{'text': a.text, 'href': a.get('href')} for a in soup.find_all('a')]
        }
        
        # Salva em JSON
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"scraping_result_{timestamp}.json"
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
            
        return data, filename
    
    except Exception as e:
        print(f"Erro durante o scraping: {str(e)}")
        return None, None

## 2. Captura de Tela

Função para capturar screenshots.

In [ ]:
def capture_screen(region=None):
    """Captura a tela ou uma região específica."""
    try:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"screenshot_{timestamp}.png"
        
        screenshot = pyautogui.screenshot(region=region)
        screenshot.save(filename)
        
        return filename
    except Exception as e:
        print(f"Erro na captura: {str(e)}")
        return None

## 3. Geração de PDF

Função para criar PDFs a partir dos dados coletados.

In [ ]:
def create_pdf(filename, content):
    """Cria um PDF com o conteúdo fornecido."""
    try:
        doc = SimpleDocTemplate(filename, pagesize=letter)
        styles = getSampleStyleSheet()
        story = []

        # Título
        title_style = ParagraphStyle(
            'CustomTitle',
            parent=styles['Heading1'],
            fontSize=24,
            spaceAfter=30
        )
        story.append(Paragraph(content.get('title', 'Documento'), title_style))
        story.append(Spacer(1, 12))

        # Parágrafos
        for paragraph in content.get('paragraphs', []):
            story.append(Paragraph(paragraph, styles['Normal']))
            story.append(Spacer(1, 12))

        # Links
        if content.get('links'):
            story.append(Paragraph('Links:', styles['Heading2']))
            for link in content['links']:
                story.append(Paragraph(
                    f"{link['text']}: {link['href']}", 
                    styles['Normal']
                ))
                story.append(Spacer(1, 6))

        doc.build(story)
        return filename

    except Exception as e:
        print(f"Erro ao criar PDF: {str(e)}")
        return None

## 4. Exemplo de Uso

Vamos testar as funções criadas.

In [ ]:
# URL de exemplo
url = "https://docs.agentql.com/examples/python-examples"

# Realiza web scraping
print("1. Realizando web scraping...")
data, json_file = web_scraping(url)
if data:
    print(f"Dados salvos em: {json_file}")
    print(f"Título encontrado: {data['title']}")
    print(f"Número de parágrafos: {len(data['paragraphs'])}")
    print(f"Número de links: {len(data['links'])}")

# Captura screenshot
print("\n2. Capturando screenshot...")
screenshot_file = capture_screen()
if screenshot_file:
    print(f"Screenshot salvo como: {screenshot_file}")

# Gera PDF
if data:
    print("\n3. Gerando PDF...")
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    pdf_file = f"report_{timestamp}.pdf"
    if create_pdf(pdf_file, data):
        print(f"PDF gerado com sucesso: {pdf_file}")